<a href="https://colab.research.google.com/github/Ravindra1972/Anaytics-in-finance-using-Python/blob/main/CAPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf

In [3]:
tickers = ['SBIN.NS', 'TCS.NS', 'INFY.NS', 'BHEL.NS', 'CIPLA.NS', 'ITC.NS', '^NSEI']
stock_data = yf.download(tickers, start="2016-08-1", end="2023-11-1")

[*********************100%%**********************]  7 of 7 completed


In [5]:
rf = 0.01307
daily_returns = stock_data['Adj Close'].pct_change().dropna()
rm = daily_returns['^NSEI'].mean() * 252


In [10]:
cov = daily_returns.cov()
var = daily_returns['^NSEI'].var()
beta_tcs = cov.loc['TCS.NS', '^NSEI']/ var

In [11]:
r_tcs = rf + (beta_tcs * (rm-rf))
print("Expected rate of return on TCS is: " + str(r_tcs))

Expected rate of return on TCS is: 0.0885062187181781


In [12]:
beta = cov.loc['^NSEI']/ var
r_stocklist = rf + (beta * (rm-rf))
print(r_stocklist)

BHEL.NS     0.151397
CIPLA.NS    0.066228
INFY.NS     0.104531
ITC.NS      0.096435
SBIN.NS     0.159349
TCS.NS      0.088506
^NSEI       0.126345
Name: ^NSEI, dtype: float64
